<a href="https://colab.research.google.com/github/Sachin961120/OptimizeCCNModel/blob/main/OptimizeCCNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 102kB 8.7MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=2baafb683539138610a4b0ea19440a6c59e67a92bf0127ae453a540008349fb9
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels),(test_images, test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images= train_images/255.0
test_images= test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [8]:
train_images[0].shape

(28, 28, 1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tunner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='Mnist_Fashion' )

In [15]:
tunner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 6 Complete [00h 00m 41s]
val_accuracy: 0.9056666493415833

Best val_accuracy So Far: 0.9135000109672546
Total elapsed time: 00h 03m 59s
INFO:tensorflow:Oracle triggered exit


In [18]:
model=tunner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1239120   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,317,722
Trainable params: 1,317,722
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1480 - accuracy: 0.9445 - val_loss: 0.2415 - val_accuracy: 0.9175
Epoch 5/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1122 - accuracy: 0.9583 - val_loss: 0.2593 - val_accuracy: 0.9208
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0810 - accuracy: 0.9704 - val_loss: 0.3072 - val_accuracy: 0.9165
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0610 - accuracy: 0.9767 - val_loss: 0.3298 - val_accuracy: 0.9158
Epoch 8/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0484 - accuracy: 0.9823 - val_loss: 0.4025 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0389 - accuracy: 0.9858 - val_loss: 0.4344 - val_accuracy: 0.9158
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0315 - accuracy: 0.9889 - val_loss: 0.4581 - val_accuracy